# "Binary Prediction of Smoker Status using Bio-Signals" Competition

Submission by: Atul Parida

## Overview
This notebook contains the code and analysis for the "Binary Prediction of Smoker Status using Bio-Signals" competition on Kaggle. In this competition, the goal is to build a machine learning model that predicts whether given biodata belongs to a smoker or not. We will explore the dataset, preprocess the text data, build and evaluate models, and make predictions.

## Table of Contents
1. [Data Exploration](#data-exploration)
2. [Data Preprocessing](#data-preprocessing)
3. [Model Building](#model-building)
4. [Model Evaluation](#model-evaluation)
5. [Results and Conclusion](#results-and-conclusion)


### Data Exploration

In [1]:
### Essential imports ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [2]:
train_data_path = os.path.join(os.getcwd(), 'data', 'train.csv')
test_data_path = os.path.join(os.getcwd(), 'data', 'test.csv')
output_path = os.path.join(os.getcwd(), 'output', 'output.csv')

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [3]:
train_data.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1


### Data Preprocessing

### Model Building

### Model Evaluation

### Results and Conclusions